In [1]:
import pandas as pd
import numpy as np

In [2]:
nombre_archivo = 'precios_semana_20200503.json'
# este nombre tiene que tomarse de la ejecución del script; para desarrollarlo lo hardcodeamos
extension = nombre_archivo.split('.')[1]
# guarda la extension del archivo
ruta = './Datasets/'
# harcodeamos ruta relativa
codfecha = (nombre_archivo.split('.')[0])[-8:]
fecha_carga = codfecha[0:4]+'/'+codfecha[4:6]+'/'+codfecha[6:8]
dfprecio = pd.read_json(str(ruta)+str(nombre_archivo))

In [3]:
dfprecio.head()

,precio,producto_id,sucursal_id
0,29.9,0000000002288,2-1-187
1,39.9,0000000002288,2-3-247
2,499.99,0000000205870,9-1-685
3,539.99,0000000205870,9-2-22
4,519.99,0000000205870,9-2-59


In [4]:
dfprecio.describe(include='all')

,precio,producto_id,sucursal_id
count,397734,397734,397734
unique,19617,61505,170
top,119,7793253001926,50-1-2
freq,2400,90,15134


In [5]:
def sacar_espacios(df):
    '''
    Quitar espacios vacíos de de ambos lados de las cols con strings.
    '''
    sacar_espacios = lambda x: x.strip() if isinstance(x, str) else x
    return df.applymap(sacar_espacios)
sacar_espacios(dfprecio)

,precio,producto_id,sucursal_id
0,29.9,0000000002288,2-1-187
1,39.9,0000000002288,2-3-247
2,499.99,0000000205870,9-1-685
3,539.99,0000000205870,9-2-22
4,519.99,0000000205870,9-2-59
...,...,...,...
397729,139.99,9569753142128,25-1-1
397730,34.99,9795403001143,25-1-1
397731,312.5,9990385651922,5-1-5
397732,312.5,9990385651939,5-1-5


In [6]:
dfprecio['precio']=np.where(dfprecio['precio']=='',0,dfprecio['precio'])

In [7]:
dfprecio['producto_id']=dfprecio['producto_id'].astype(str)
dfprecio['sucursal_id']=dfprecio['sucursal_id'].astype(str)

In [8]:
dfprecio['precio']=dfprecio['precio'].astype(float)

In [9]:
dfprecio[dfprecio['precio'].isnull()|dfprecio['producto_id'].isnull()|dfprecio['sucursal_id'].isnull()]

,precio,producto_id,sucursal_id


In [10]:
dfprecio[dfprecio['producto_id'].isnull()]

,precio,producto_id,sucursal_id


In [11]:
#elimino registros sin codigo de producto.
#registros sin codigo no me van a servir para ningun analisis
lista_sin_cod_prod = dfprecio[dfprecio['sucursal_id'].isnull()].index.tolist()


In [12]:
dfprecio.drop(lista_sin_cod_prod, inplace=True)

In [13]:
dfprecio.info()
#observo que solo quedan nulos en la columna 'precio'
#para salvar los nulos de la columna precio, procederemos a imputar el precio promedio de dicho producto en el resto de las sucursales.
#pero primero debemos normalizar el contenido de ambas columnas 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 397734 entries, 0 to 397733
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   precio       397734 non-null  float64
 1   producto_id  397734 non-null  object 
 2   sucursal_id  397734 non-null  object 
dtypes: float64(1), object(2)
memory usage: 9.1+ MB


In [14]:
# comencemos con la columna producto_id
# creo una columna 'cant' que indique la cantidad de caracteres
dfprecio['cant']=dfprecio['producto_id'].apply(lambda x: len(str(x)))

In [15]:
dfprecio['cant'].unique()

array([13, 17, 18], dtype=int64)

In [16]:
dfprecio[dfprecio['cant']==18].head()

,precio,producto_id,sucursal_id,cant
10012,59.00,10-1-2300049000008,10-1-26,18
10013,5.10,10-1-2300073000005,10-1-142,18
10014,59.00,10-1-2300073000005,10-1-147,18
10015,59.00,10-1-2300075000003,10-1-26,18
10016,249.99,10-1-2300222000009,10-1-235,18


In [17]:




dfprecio['id2']=dfprecio['producto_id'].apply(lambda x: x[-13:])
import re
dfprecio['caractid2']=dfprecio['id2'].apply(lambda x: re.findall('\D',x))


In [18]:
dfprecio['caractid2'].describe()
#observo que no hay caracteres no numericos en el nuevo id

count     397734
unique         1
top           []
freq      397734
Name: caractid2, dtype: object

In [19]:
dfprecio.head()

,precio,producto_id,sucursal_id,cant,id2,caractid2
0,29.90,0000000002288,2-1-187,13,0000000002288,[]
1,39.90,0000000002288,2-3-247,13,0000000002288,[]
2,499.99,0000000205870,9-1-685,13,0000000205870,[]
3,539.99,0000000205870,9-2-22,13,0000000205870,[]
4,519.99,0000000205870,9-2-59,13,0000000205870,[]


In [20]:

dfprecio.pop('cant')
dfprecio.pop('id2')


0         0000000002288
1         0000000002288
2         0000000205870
3         0000000205870
4         0000000205870
              ...      
397729    9569753142128
397730    9795403001143
397731    9990385651922
397732    9990385651939
397733    9990385651946
Name: id2, Length: 397734, dtype: object

In [21]:

dfprecio.pop('caractid2')


0         []
1         []
2         []
3         []
4         []
          ..
397729    []
397730    []
397731    []
397732    []
397733    []
Name: caractid2, Length: 397734, dtype: object

In [22]:

dfprecio.head()

,precio,producto_id,sucursal_id
0,29.90,0000000002288,2-1-187
1,39.90,0000000002288,2-3-247
2,499.99,0000000205870,9-1-685
3,539.99,0000000205870,9-2-22
4,519.99,0000000205870,9-2-59


In [23]:
#normalizo sucursal
dfprecio_aux=dfprecio['sucursal_id'].str.split('-', expand=True)

In [24]:
dfprecio_aux[0]=dfprecio_aux[0].astype(int)
dfprecio_aux[1]=dfprecio_aux[1].astype(int)
dfprecio_aux[2]=dfprecio_aux[2].astype(int)


In [25]:
dfprecio_aux.head()
dfprecio_aux.describe()

,0,1,2
count,397734.000000,397734.000000,397734.000000
mean,17.113138,1.342649,440.908094
std,15.180101,0.673551,1196.168912
min,1.000000,1.000000,1.000000
25%,9.000000,1.000000,7.000000
50%,10.000000,1.000000,108.000000
75%,20.000000,1.000000,233.000000
max,65.000000,6.000000,30977.000000


In [26]:
dfprecio_aux['sucursal_id']=dfprecio_aux[0].astype(str)+'-'+dfprecio_aux[1].astype(str)+'-'+dfprecio_aux[2].astype(str)

In [27]:
dfprecio_aux.head()

,0,1,2,sucursal_id
0,2,1,187,2-1-187
1,2,3,247,2-3-247
2,9,1,685,9-1-685
3,9,2,22,9-2-22
4,9,2,59,9-2-59


In [28]:
dfprecio['sucursal_id']=dfprecio_aux['sucursal_id']

In [29]:
dfprecio['fecha']=fecha_carga

In [30]:
dfprecio.head()

,precio,producto_id,sucursal_id,fecha
0,29.90,0000000002288,2-1-187,2020/05/03
1,39.90,0000000002288,2-3-247,2020/05/03
2,499.99,0000000205870,9-1-685,2020/05/03
3,539.99,0000000205870,9-2-22,2020/05/03
4,519.99,0000000205870,9-2-59,2020/05/03


In [31]:
dfprecio.rename(columns={'producto_id': 'Id_Producto', 'sucursal_id': 'Id_Sucursal'}, inplace=True)

In [32]:
dfprecio.insert(0, 'Id', 0)

In [33]:
#exportacion a csv de la tabla
#from encodings import utf_8


ruta_export = './Data_out/'
nombre_archivo_out = 'producto_precio_' + str(codfecha) + '.csv'
dfprecio.to_csv(ruta_export+nombre_archivo_out, index=False, encoding='utf-8-sig')